In [1]:
from pyspark.sql import SparkSession
from pyspark.sql.types import DoubleType, FloatType, LongType, StructType,StructField, StringType
from pyspark.sql import functions as F

In [2]:
spark = SparkSession.builder.appName("Jupyter").getOrCreate()

25/09/17 15:33:51 WARN SparkSession: Using an existing Spark session; only runtime SQL configurations will take effect.


In [3]:
spark

In [4]:
spark.sql("SHOW NAMESPACES IN demo").show()

+----------------+
|       namespace|
+----------------+
|retail_analytics|
+----------------+



In [5]:
spark.sql("SHOW CURRENT NAMESPACE").show()

+-------+----------------+
|catalog|       namespace|
+-------+----------------+
|   demo|retail_analytics|
+-------+----------------+



In [6]:
spark.sql("SHOW TABLES IN demo.retail_analytics").show()

+----------------+---------------+-----------+
|       namespace|      tableName|isTemporary|
+----------------+---------------+-----------+
|retail_analytics|   dim_customer|      false|
|retail_analytics|       dim_date|      false|
|retail_analytics|    dim_product|      false|
|retail_analytics|     dim_seller|      false|
|retail_analytics|fct_order_items|      false|
|retail_analytics|     fct_orders|      false|
|retail_analytics|stg_order_items|      false|
|retail_analytics|     stg_orders|      false|
+----------------+---------------+-----------+



In [26]:
df = spark.read.csv("s3a://csv-input/customers.csv", header=True)

25/09/17 11:53:04 WARN MetricsConfig: Cannot locate configuration: tried hadoop-metrics2-s3a-file-system.properties,hadoop-metrics2.properties


In [7]:
df = spark.sql("SELECT * FROM fct_order_items WHERE  order_date > '2024-11-01'")

In [8]:
df.printSchema()

root
 |-- order_id: string (nullable = true)
 |-- order_item_id: integer (nullable = true)
 |-- product_id: string (nullable = true)
 |-- seller_id: string (nullable = true)
 |-- item_value: double (nullable = true)
 |-- freight_value: double (nullable = true)
 |-- order_date: date (nullable = true)
 |-- shipping_limit_date: date (nullable = true)



In [9]:
df.show(10)

+--------+-------------+----------+---------+----------+-------------+----------+-------------------+
|order_id|order_item_id|product_id|seller_id|item_value|freight_value|order_date|shipping_limit_date|
+--------+-------------+----------+---------+----------+-------------+----------+-------------------+
|00000184|            2|  00000059| 00000022|      93.6|        26.09|2024-12-17|               NULL|
|00000184|            0|  00000039| 00000022|     117.0|        16.99|2024-12-17|               NULL|
|00000184|            4|  00000096| 00000022|     128.8|        20.23|2024-12-17|               NULL|
|00000184|            1|  00000096| 00000022|     128.8|        28.76|2024-12-17|               NULL|
|00000184|            3|  00000040| 00000022|     138.0|        30.44|2024-12-17|               NULL|
|00000259|            1|  00000008| 00000191|     101.2|        12.85|2024-12-20|               NULL|
|00000259|            4|  00000042| 00000191|      66.0|        20.57|2024-12-20| 

In [10]:
spark.stop()